Vector Search Qdrant

## 1. Importación de librerías y conexión con Qdrant

Primero asegúrate de correr e instalar las dependencias necesarias
Y dejar el contenedor corriendo. <p>
Setup: [set_up_qdrant]()

Importación de librerías
- Clase `QdrantClient`: permite establecer conexión con el servicio de Qdrant.
- Módulo `models`: permite establecer configuraciones y parámetros necesarios.

In [1]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Inicializamos el cliente
client = QdrantClient("http://localhost:6333")

## 2. Dataset


Hay que entender el dataset, conocer principalmente:
- Qué tipo de contenido es? imagen, video, texto o combinación
- Especificar: Si es texto, entonces qué lenguaje es, contiene caracteres especiales?

¿Porqué es necesario responder estas preguntas?
Para definir:
- El tipo de esquema para los datos. (¿qué es lo se va a vectorizar?, ¿Qué se va a almacenar en la metadata?)
- El modelo correcto del embedding. Tenemos que evaluar diferentes parámetros como el dominio, la precisión y los recursos requeridos           

In [3]:
# Importación del dataset
# pip install requests
import requests

ruta_doc= 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_rpta= requests.get(ruta_doc)
documentos = docs_rpta.json()
# documentos

In [4]:
print(f"Tamaño de documentos:{len(documentos)}")
print(f"Data type de documentos:{type(documentos)}")
print(f"Data type de dato del primer elemento de documentos:{type(documentos[0])}")

print('\nCursos del FaQ Zoomcamp:')
for i in documentos:
    print(f"- {i['course']}")

Tamaño de documentos:3
Data type de documentos:<class 'list'>
Data type de dato del primer elemento de documentos:<class 'dict'>

Cursos del FaQ Zoomcamp:
- data-engineering-zoomcamp
- machine-learning-zoomcamp
- mlops-zoomcamp


Entonces, después de analizar `documentos`, la información relevante es:
- Tamaño: 3 elementos
- Lista de diccionarios

Estructura de `documentos` <p>

```git bash
[
    {'course':'data-engineering-zoomcamp',
    'documents': [{'text': ' ' , 'section':' '  , 'question':' '  },
                  {'text': ' ' , 'section':' '  , 'question':' '  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    
    }, 
    {'course':'machine-learning-zoomcamp',
    'documents': [{'text':  , 'section':  , 'question':  },
                  {'text':  , 'section':  , 'question':  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    
    }, 
    {'course':'mlops-zoomcamp',
    'documents': [{'text':  , 'section':  , 'question':  },
                  {'text':  , 'section':  , 'question':  },
                                 ...                    
                {'text':  , 'section':  , 'question':  } ]
    } 
]
```

Cada elemento de la lista es un diccionario, que contiene 2 elementos(course y documents). <p>
Del cual documents es también una lista de diccionarios, donde cada diccionario contiene 3 elementos(text, section, question).

1. Verificamos que la data se encuentra limpia y fragmentada (chunk, divide la data en pequeñas partes porque es más fácil para los modelos de embeddings procesarla)
2. Tenemos que definir
    - Los campos(fields) para semantic_search
    - Campos para almacenar como *metadata*, pueden ser usados como filtros 

3. Se definen: 
    - Campos para semantic search: question, text
    - Campos para metadata(también como filtros): course, section

## 3. Elección del modelo de embeddings

Los modelos de embeddings como ya lo hemos mencionado permite convertir la data en vectores. Para escoger un 'buen' modelo, depende ciertos factores:
- La tarea, el tipo de dato y sus características. (texto, inglés )
- La evaluación entre la 'precisión de búsqueda' y los 'recursos usados' (embeddings más grandes requieren más almacenamiento y memoria)
- El costo de deducir(o inferir) 
etc

La mejor manera de escoger el modelo es testear diferentes opciones en tu propia data. <p>
En este caso, vamos a utilizar FastEmbed. <p>
Documentación: [FastEmbed](https://github.com/qdrant/fastembed).


In [5]:
from fastembed import TextEmbedding

# modelos disponibles
modelos= TextEmbedding.list_supported_models()

# primeros 5 modelos
modelos[:5] 

# modelo_embedding= TextEmbedding()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

Es lógico que nuestro modelo no produzca una alta dimensionalidad para evitar usar recursos de más, por lo tanto nos declinamos por una moderada dimensionalidad (Ej: 512 dimensionalidades)

In [6]:
# para formatear la salida con indentacion
import json

dimension= 512
for i in modelos:
    if i['dim'] == dimension:
        print (json.dumps(i, indent=2))



{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

El modelo 1: `BAAI/bge-small-zh-v1.5`
- Para texto pero para lenguaje Chino, descartado

El modelo 2: `Qdrant/clip-ViT-B-32-text`
- Multimoda, Para texto e imágenes, pero no es necesario impágenes, descartado

El modelo 3: `jinaai/jina-embeddings-v2-small-en`
- El más adecuado descartando los previos
- [Documentación del modelo usado para embedding](https://huggingface.co/jinaai/jina-embeddings-v2-small-en)

In [7]:
modelo_seleccionado= "jinaai/jina-embeddings-v2-small-en"

# Este modelo como muchos, fue entrenado usando similitud de coseno

Ahora ya estamos listos para configurar los parámetros del modelo con **Qdrant** 

## 4. Configuración con Qdrant

### Creación de una colección:
Debemos definir:
- Nombre
- Configuración del vector: 
    -  Size, dimensionalidad del vector
    - Distance Metric: el método para medir la similitud entre vectores
    Tipo de métodos: dot, cosine Euclid, manhatthan

In [9]:
# from qdrant_client import QdrantClient, models
# client = QdrantClient(url="http://localhost:6333")

nombre_coleccion= 'zoomcamp-rag'
client.create_collection(
    collection_name= nombre_coleccion, 
    vectors_config=models.VectorParams(
        size= dimension, 
        distance= models.Distance.COSINE
    )
)

True

### Crear e insertar Points en la Colección
Recordar que la definción de un Point es `P(ID, vector, payload(opcional))`
Explicación más detallada en [set_up_qdrant](02_Week_vector_search/set_up_qdrant.md)

In [12]:
modelo_seleccionado

'jinaai/jina-embeddings-v2-small-en'

Creación de puntos

In [13]:
# para almacenar los puntos
points= []
id=0 #por defecto el identificador de los puntos 0 

for i in documentos:
    for j in i['documents']: # text-section -question
        # print(j)
        point = models.PointStruct(
            id= id, 
            vector= models.Document(text= j['text'], model= modelo_seleccionado), 
            payload= {
                "text": j['text'], 
                "section":j['section'], 
                "course": i['course']
            }
        )
        points.append(point)
        id +=1

In [15]:
points[:4]

[PointStruct(id=0, vector=Document(text="The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to registe

Finalmente se resume este notebook para un sistema Rag con Vector Search. 
[colocar el notebook de rag]()